In [4]:
%matplotlib inline

import mne
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

from mne.time_frequency import tfr_morlet, psd_multitaper, AverageTFR
from mne.connectivity import spectral_connectivity, seed_target_indices

import os
os.environ['ETS_TOOLKIT'] = 'qt4'
os.environ['QT_API'] = 'pyqt'
%gui qt

In [5]:
fif = mne.read_epochs('data/epochs/01_short-epo.fif')
picks = mne.pick_types(fif.info, eeg=True, stim=False)

Reading data/epochs/01_short-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 62)  idle
    Found the data of interest:
        t =    -500.00 ...    2100.00 ms
        0 CTF compensation matrices available
52 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
52 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [6]:
path = 'data/con_median_NEW/'
cfile = '_S1_median_alpha000-con'
footer = '.csv'
fname = path + cfile + footer
con = np.loadtxt(fname, delimiter=',', skiprows=1)
# con[con > 0] = 0
con = abs(con)

# the epochs contain an EOG channel, which we remove now
ch_names = fif.ch_names[:62]
idx = [ch_names.index(name) for name in ch_names]
con = con[idx][:, idx]

# Now, visualize the connectivity in 3D
from mayavi import mlab  # noqa
mlab.figure(size=(700, 800), bgcolor=(1, 1, 1))

# Plot the sensor locations
sens_lay = mne.find_layout(fif.info).pos[:,:2]
zero = np.ones(sens_lay.shape[0])
sens_loc = np.c_[sens_lay, zero]
sens_loc.shape

pts = mlab.points3d(sens_loc[:, 0], sens_loc[:, 1], sens_loc[:, 2],
                    color=(0.4, 0.4, 0.4), opacity=1, scale_factor=0.025)

# # Get the strongest connections
n_con = np.count_nonzero(con)
min_dist = 0  # exclude sensors that are less than 5cm apart
threshold = np.sort(con, axis=None)[-n_con]
ii, jj = np.where(con >= threshold)

# Remove close connections
con_nodes = list()
con_val = list()

for i, j in zip(ii, jj):
    if linalg.norm(sens_loc[i] - sens_loc[j]) > min_dist:
        con_nodes.append((i, j))
        con_val.append(con[i, j])

con_val = np.array(con_val)

# Show the connections as tubes between sensors
# vmax = np.max(con_val)
# memory task = 0.0-0.3, flash task = 0.0 - 0.8
vmax = 0.8
# vmin = np.min(con_val)
vmin = 0

    
#print(np.sqrt(((x1-x2)-(y1-y2))*((x1-x2)-(y1-y2)))

for val, nodes in zip(con_val, con_nodes):
    x1, y1, z1 = sens_loc[nodes[0]]
    x2, y2, z2 = sens_loc[nodes[1]]
    
    print(np.sqrt((x1-x2)*(x1-x2)+(y1-y2)*(y1-y2)))
            
    points = mlab.plot3d([x1, x2], [y1, y2], [z1, z2], [val, val],
                         vmin=vmin, vmax=vmax, tube_radius=0.006,
                         colormap='summer')
    points.module_manager.scalar_lut_manager.reverse_lut = True
    
    # Positive: YlOrRd
    # Negative: YlGnBu
    # corr: YlGn
    
# mlab.scalarbar(title='Weighted Phase Lag Index (WPLI)', nb_labels=5, orientation='vertical')

# Add the sensor names for the connections shown
nodes_shown = list(set([n[0] for n in con_nodes] + [n[1] for n in con_nodes]))

for node in nodes_shown:
    x, y, z = sens_loc[node]
    mlab.text3d(x, y, z, fif.ch_names[picks[node]], scale=0.02,
                color=(0, 0, 0))
    
view = (0, 0, 1)
mlab.show()

0.696337569263
0.633665270341
0.391690663855
0.462025389594
0.824585321007
0.295095493004
0.499674682278
0.853477396548
0.345719468225
0.64130209714
